# Test

In [1]:
import sys

In [2]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # view the path and verify

['../', '/Users/federicosiciliano/Desktop/Py_utils/torch_utils/ntb', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages']


In [3]:
import exp_utils, data_utils, torch_utils

## Configuration

In [4]:
cfg = exp_utils.cfg.load_configuration("config_nn")

In [5]:
cfg

{'__exp__': {'name': 'prova',
  'project_folder': '../',
  'key_len': 16,
  'key_prefix': '',
  '__imports__': ['torchvision'],
  '__nosave__': {'model.loader_params.num_workers': None,
   'model.trainer_params.accelerator': None,
   'model.trainer_params.enable_checkpointing': None}},
 'data': {'name': 'MNIST',
  'source': 'torchvision',
  'merge_before_split': False,
  'split_keys': {'train_x': ['train_x', 'val_x'],
   'train_y': ['train_y', 'val_y']},
  'test_sizes': [0.2],
  'split_random_state': 21094,
  'one_hot_encode': True,
  'scaling': 'MinMax'},
 'model': {'name': 'resnet18',
  'torchvision_params': {'weights': [None, <enum 'ResNet18_Weights'>]},
  'optimizer': {'name': 'Adam', 'params': {'lr': 0.1, 'weight_decay': 0.0005}},
  'loss': 'CrossEntropyLoss',
  'loader_params': {'batch_size': 256, 'num_workers': 4},
  'trainer_params': {'max_epochs': 2,
   'callbacks': [{'EarlyStopping': {'monitor': 'val_loss',
      'mode': 'min',
      'patience': 1}},
    {'ModelCheckpoint': {

In [6]:
for app in cfg.sweep("model.torchvision_params.weights"):
    #print(app)
    print(cfg.model)
#print(cfg)

{'name': 'resnet18', 'torchvision_params': {'weights': None}, 'optimizer': {'name': 'Adam', 'params': {'lr': 0.1, 'weight_decay': 0.0005}}, 'loss': 'CrossEntropyLoss', 'loader_params': {'batch_size': 256, 'num_workers': 4}, 'trainer_params': {'max_epochs': 2, 'callbacks': [{'EarlyStopping': {'monitor': 'val_loss', 'mode': 'min', 'patience': 1}}, {'ModelCheckpoint': {'save_top_k': 1, 'save_last': True, 'monitor': 'val_loss', 'mode': 'min', 'dirpath': '..//out/models/', 'filename': 'best'}, '__nosave__': ['ModelCheckpoint.dirpath', 'ModelCheckpoint.filename']}], 'accelerator': 'cpu', 'enable_checkpointing': True}}
{'name': 'resnet18', 'torchvision_params': {'weights': <enum 'ResNet18_Weights'>}, 'optimizer': {'name': 'Adam', 'params': {'lr': 0.1, 'weight_decay': 0.0005}}, 'loss': 'CrossEntropyLoss', 'loader_params': {'batch_size': 256, 'num_workers': 4}, 'trainer_params': {'max_epochs': 2, 'callbacks': [{'EarlyStopping': {'monitor': 'val_loss', 'mode': 'min', 'patience': 1}}, {'ModelChec

In [7]:
data = data_utils.data.load_data(**cfg["data"])

In [8]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

In [9]:
loaders = torch_utils.preparation.prepare_data_loaders(data, cfg["model"]["loader_params"])

In [10]:
main_module = torch_utils.model.get_torchvision_model(**cfg["model"])

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
callbacks = torch_utils.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
already_defined = ["callbacks"]
trainer = torch_utils.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks)

Global seed set to 42


AttributeError: module 'pytorch_lightning.callbacks' has no attribute '__nosave__'

In [ ]:
loss = torch_utils.preparation.prepare_loss(cfg["model"]["loss"])

In [ ]:
optimizer = torch_utils.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

In [ ]:
model = torch_utils.process.create_model(main_module, loss, optimizer)

In [ ]:
torch_utils.process.train_model(trainer, model, loaders)

NameError: name 'trainer' is not defined

In [ ]:
torch_utils.process.test_model(trainer, model, loaders)

NameError: name 'trainer' is not defined